In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import numpy as np

In [3]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')


def cosine_sim(row):
    tfidf = vectorizer.fit_transform([row.input, row.text])
    return ((tfidf * tfidf.T).A)[0,1]


In [10]:
df = pd.read_csv('webapp/dataset/tweets.csv', index_col=0)
df = df.drop(columns=['date', 'author','id','link','retweet'])
df.index.name = "index"
df.to_csv(r'webapp/dataset/tweets_clean.csv')
df.head(-5)

,text
index,
0,Here is my statement.pic.twitter.com/WAZiGoQqMQ
1,Is this really America? Terrible!pic.twitter.c...
2,The media and establishment want me out of the...
3,Certainly has been an interesting 24 hours!
4,Debate polls look great - thank you!\n#MAGA #A...
...,...
17206,"""When the achiever achieves, it's not a platea..."
17207,"Enter the ""Think Like A Champion"" signed book ..."
17208,"""Strive for wholeness and keep your sense of w..."


In [68]:
def sorting_tweet (input_user,df): 
    
    # Introducing data in the dataframe as a new column
    df['input'] = input_user
    
    # Applying cosine_sim function to a new column 
    df['similarity'] = df.apply(cosine_sim, axis=1)
    
    # Sorting dataframe by similarity highest value
    df = df.sort_values(by='similarity', ascending=False)
    
    # Reseting the index to get dataframe indexed after sorted
    df = df.reset_index(drop=True)
    
    # Creating output list appending the top 20 tweets
    output = []
    for i in range(0, 20): 
        output.append((df['text'][i])) 
        
    return output

In [62]:
web_input = "Hello guys, I love you"

output = sorting_tweet(web_input,df)

In [67]:
for i in range(0, 20): 
    print(output[i] + '\n')

"@UmerRizwan2: @realDonaldTrump Can i get a hello for all the Canadians down in Ottawa, Ontario ?"  Hello Umer, you sound like a great guy!

"@Candynecklace2: @realDonaldTrump I love love love Mr. Trump he is my hero " Thanks.

"@DharmaBum77: Donald Trump loves America! He loves Americans! He loves our Vets! #TrumpSupporters @realDonaldTrumppic.twitter.com/NgcoHRLj2M

.@IamStevenT stopped by my office to say hello- a great guy! http://bit.ly/1n1KP1Z 

@RealKyleMorris  You will love it!

@BrikMillerEDG You will love it!

.@williebosshog such an honor to get your endorsement. You are a fantastic guy! It will not be forgotten. Don and Eric say hello!

Paul Teutul Sr. is a fantastic guy. Although I fired him on #CelebApprentice, we will remain great friends. I love the bike he made for me.

Paul Teutul Sr. is a fantastic guy. Although I fired him on #CelebApprentice, we will remain great friends. I love the bike he made for me.

"I love America. And when you love something you protect it p